In [1]:
transformation='JW'
# transformation='BK'

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit import Aer

In [3]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
## HAMILTONIAN start

Molecule = 'H2'
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-3g'

# Molecule = 'LiH'
# geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., 1.44))]
# basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian_PySCF(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation=transformation)
### HAMILTONIAN end

#####################################

print(QubitHamiltonian)


fci_energy = Hamilt.molecule.fci_energy
print(fci_energy)

(-0.09706626816762878+0j) [] +
(-0.045302615503799264+0j) [X0 X1 Y2 Y3] +
(0.045302615503799264+0j) [X0 Y1 Y2 X3] +
(0.045302615503799264+0j) [Y0 X1 X2 Y3] +
(-0.045302615503799264+0j) [Y0 Y1 X2 X3] +
(0.1714128264477689+0j) [Z0] +
(0.16868898170361207+0j) [Z0 Z1] +
(0.12062523483390414+0j) [Z0 Z2] +
(0.1659278503377034+0j) [Z0 Z3] +
(0.17141282644776892+0j) [Z1] +
(0.1659278503377034+0j) [Z1 Z2] +
(0.12062523483390414+0j) [Z1 Z3] +
(-0.22343153690813564+0j) [Z2] +
(0.1744128761226159+0j) [Z2 Z3] +
(-0.22343153690813558+0j) [Z3]
-1.137283834488502


From Helgaker, T., P. Jorgensen,and J. Olsen (2014),Molecularelectronic-structure theory(John Wiley & Sons)
we known

$$H_{2}^{ground} = 0.9939| 1100\rangle - 0.1106| 0011\rangle$$

From group theory we know only double excitation terms are important!
We can see this from our ijab operators that qubits 1 and 3 are only acted on by $Z$ therefore experience no population changes... Can even see this from the answer if we write it in BK basis:

$$H_{2}^{BK} = 0.9939| 1000\rangle - 0.1106| 0010\rangle$$

See how qubits 1 and 3 always remain at 0!

therefore can **remove** these measurements from Hamiltonian... as we know what expectation values they should give!

In the case for this Hamiltonian only $I$ and $Z$ act on these qubits! So even easier to remove (expectation values = +1 (as act on $| 0\rangle$ state!), so just remove and add to relivent H terms!

LOOK at:

PHYS. REV. X, **8**, 031022 (2018)

In [4]:
from quchem.Ansatz_Generator_Functions import *
n_electrons=Hamilt.molecule.n_electrons
n_qubits=Hamilt.molecule.n_qubits

ansatz_obj = Ansatz(n_electrons,n_qubits)
print('JW ground state = ', ansatz_obj.Get_JW_HF_state_in_OCC_basis())
print('BK ground state = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())

JW ground state =  [1, 1, 0, 0]
BK ground state =  [1. 0. 0. 0.]


In [5]:
qubits_to_remove = Find_I_Z_indices_in_Hamiltonian(QubitHamiltonian, Hamilt.molecule.n_qubits)
print('qubits only acted on by I or Z:', qubits_to_remove)


input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
# Remove_Z_terms_from_Hamiltonian
NewQubitHamiltonian = Remove_Z_terms_from_Hamiltonian(
                                                    QubitHamiltonian, 
                                                    input_state,
                                                    qubits_to_remove,
                                                    check_reduction=True)
NewQubitHamiltonian

qubits only acted on by I or Z: []


(-0.09706626816762878+0j) [] +
(-0.045302615503799264+0j) [X0 X1 Y2 Y3] +
(0.045302615503799264+0j) [X0 Y1 Y2 X3] +
(0.045302615503799264+0j) [Y0 X1 X2 Y3] +
(-0.045302615503799264+0j) [Y0 Y1 X2 X3] +
(0.1714128264477689+0j) [Z0] +
(0.16868898170361207+0j) [Z0 Z1] +
(0.12062523483390414+0j) [Z0 Z2] +
(0.1659278503377034+0j) [Z0 Z3] +
(0.17141282644776892+0j) [Z1] +
(0.1659278503377034+0j) [Z1 Z2] +
(0.12062523483390414+0j) [Z1 Z3] +
(-0.22343153690813564+0j) [Z2] +
(0.1744128761226159+0j) [Z2 Z3] +
(-0.22343153690813558+0j) [Z3]

In [6]:
# # MANUAL LiH BK red
# qubits_to_remove=[0,1,8,9,10,11]
# print('qubits only acted on by I or Z:', qubits_to_remove)


# input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
# # Remove_Z_terms_from_Hamiltonian
# NewQubitHamiltonian = Remove_Z_terms_from_Hamiltonian(
#                                                     QubitHamiltonian, 
#                                                     input_state,
#                                                     qubits_to_remove,
#                                                     check_reduction=False)
# NewQubitHamiltonian

In [7]:
qubitNo_re_label_dict, NewQubitHamiltonian_relabelled = Re_label_Hamiltonian(NewQubitHamiltonian)
NewQubitHamiltonian_relabelled

(-0.09706626816762878+0j) [] +
(-0.045302615503799264+0j) [X0 X1 Y2 Y3] +
(0.045302615503799264+0j) [X0 Y1 Y2 X3] +
(0.045302615503799264+0j) [Y0 X1 X2 Y3] +
(-0.045302615503799264+0j) [Y0 Y1 X2 X3] +
(0.1714128264477689+0j) [Z0] +
(0.16868898170361207+0j) [Z0 Z1] +
(0.12062523483390414+0j) [Z0 Z2] +
(0.1659278503377034+0j) [Z0 Z3] +
(0.17141282644776892+0j) [Z1] +
(0.1659278503377034+0j) [Z1 Z2] +
(0.12062523483390414+0j) [Z1 Z3] +
(-0.22343153690813564+0j) [Z2] +
(0.1744128761226159+0j) [Z2 Z3] +
(-0.22343153690813558+0j) [Z3]

# Find what new FCI energy is
- here should be the same as true answer!

In [8]:
from openfermion import qubit_operator_sparse
from scipy.sparse import csc_matrix
new_Molecular_H_MATRIX =  csc_matrix(qubit_operator_sparse(NewQubitHamiltonian_relabelled))

# new_Molecular_H_MATRIX = np.flip(new_Molecular_H_MATRIX)

from scipy.sparse.linalg import eigs
try:
    eig_values, eig_vectors = eigs(new_Molecular_H_MATRIX)
except:
    from scipy.linalg import eig
    eig_values, eig_vectors = eig(new_Molecular_H_MATRIX.todense())
    
new_FCI_Energy = min(eig_values)


index = np.where(eig_values==new_FCI_Energy)[0][0]
ground_state_vector = eig_vectors[:, index]

print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', fci_energy)
print(np.isclose(new_FCI_Energy, fci_energy))

new_FCI =  (-1.137283834488503+3.7121881080805557e-17j) VS old FCI: -1.137283834488502
True


In [9]:
np.dot(ground_state_vector.conj().T, new_Molecular_H_MATRIX.dot(ground_state_vector))

(-1.1372838344885015+0j)

# HF + UCCSD

In [10]:
input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
new_input_state = np.delete(input_state, qubits_to_remove)

n_qubits_new = len(new_input_state)
new_input_state

array([1, 1, 0, 0])

In [11]:
ansatz_obj.Get_ia_and_ijab_terms()

print('ia standard')
print(ansatz_obj.Sec_Quant_CC_ia_Fermi_ops)

print('ijab standard')
print(ansatz_obj.Sec_Quant_CC_ijab_Fermi_ops)

### single trotter step
ansatz_obj.UCCSD_single_trotter_step(transformation,
                                     List_FermiOps_ia=ansatz_obj.Sec_Quant_CC_ia_Fermi_ops,
                                     List_FermiOps_ijab=ansatz_obj.Sec_Quant_CC_ijab_Fermi_ops)

print('')
print('ia standard')
print(ansatz_obj.Second_Quant_CC_single_Trot_list_ia)
print('ijab standard')
print(ansatz_obj.Second_Quant_CC_single_Trot_list_ijab)

ia standard
[-1.0 [0^ 2] +
1.0 [2^ 0], -1.0 [1^ 3] +
1.0 [3^ 1]]
ijab standard
[-1.0 [0^ 1^ 2 3] +
1.0 [3^ 2^ 1 0]]

ia standard
[-0.5j [X0 Z1 Y2] +
0.5j [Y0 Z1 X2], -0.5j [X1 Z2 Y3] +
0.5j [Y1 Z2 X3]]
ijab standard
[0.125j [X0 X1 X2 Y3] +
0.125j [X0 X1 Y2 X3] +
-0.125j [X0 Y1 X2 X3] +
0.125j [X0 Y1 Y2 Y3] +
-0.125j [Y0 X1 X2 X3] +
0.125j [Y0 X1 Y2 Y3] +
-0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3]]


In [12]:
new_CC_ia_single_trot = Remove_indices_from_qubit_ops(ansatz_obj.Second_Quant_CC_single_Trot_list_ia,
                                                      qubits_to_remove)
print('ia reduced')
print(new_CC_ia_single_trot)

new_CC_ijab_single_trot = Remove_indices_from_qubit_ops(ansatz_obj.Second_Quant_CC_single_Trot_list_ijab,
                                                      qubits_to_remove)
print('ijab reduced')
print(new_CC_ijab_single_trot)


relabelled_new_CC_ia_single_trot =Re_label_qubit_operators(qubitNo_re_label_dict, new_CC_ia_single_trot)
relabelled_new_CC_ijab_single_trot =Re_label_qubit_operators(qubitNo_re_label_dict, new_CC_ijab_single_trot)

print('')
print('ia reduced relabelled')
print(relabelled_new_CC_ia_single_trot)
print('')
print('ijab reduced relabelled')
print(relabelled_new_CC_ijab_single_trot)

ia reduced
[-0.5j [X0 Z1 Y2] +
0.5j [Y0 Z1 X2], -0.5j [X1 Z2 Y3] +
0.5j [Y1 Z2 X3]]
ijab reduced
[0.125j [X0 X1 X2 Y3] +
0.125j [X0 X1 Y2 X3] +
-0.125j [X0 Y1 X2 X3] +
0.125j [X0 Y1 Y2 Y3] +
-0.125j [Y0 X1 X2 X3] +
0.125j [Y0 X1 Y2 Y3] +
-0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3]]

ia reduced relabelled
[-0.5j [X0 Z1 Y2] +
0.5j [Y0 Z1 X2], -0.5j [X1 Z2 Y3] +
0.5j [Y1 Z2 X3]]

ijab reduced relabelled
[0.125j [X0 X1 X2 Y3] +
0.125j [X0 X1 Y2 X3] +
-0.125j [X0 Y1 X2 X3] +
0.125j [X0 Y1 Y2 Y3] +
-0.125j [Y0 X1 X2 X3] +
0.125j [Y0 X1 Y2 Y3] +
-0.125j [Y0 Y1 X2 Y3] +
-0.125j [Y0 Y1 Y2 X3]]


# Ansatz Circuit

In [13]:
# for op in relabelled_new_CC_ia_single_trot[1]:
#     print(op)
# print(list(op.terms.keys())[0])

In [14]:
# UCCSD_ansatz_Q_Circ_obj = Ansatz_Circuit(new_input_state,
#                                      relabelled_new_CC_ia_single_trot, 
#                                      relabelled_new_CC_ijab_single_trot)

# theta_ia = [0 for _ in range(len(relabelled_new_CC_ia_single_trot))]
# theta_ijab = [0 for _ in range(len(relabelled_new_CC_ijab_single_trot))]

# UCCSD_ansatz_Q_Circ =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
#                                         Theta_param_list_ia=theta_ia, 
#                                          Theta_param_list_ijab=theta_ijab,
#                                          ia_first=True)
# UCCSD_ansatz_Q_Circ

# Graph

In [15]:
from tqdm.notebook import tqdm

In [16]:
Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(NewQubitHamiltonian_relabelled)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)

anti_commuting_sets

Building Graph Edges: 100%|##########| 15/15 [00:00<00:00, 1995.13it/s]


{0: [(0.1659278503377034+0j) [Z1 Z2]],
 1: [(0.12062523483390414+0j) [Z1 Z3]],
 2: [(0.1659278503377034+0j) [Z0 Z3]],
 3: [(0.12062523483390414+0j) [Z0 Z2]],
 4: [(0.16868898170361207+0j) [Z0 Z1]],
 5: [(0.1744128761226159+0j) [Z2 Z3]],
 6: [(-0.09706626816762878+0j) []],
 7: [(-0.22343153690813558+0j) [Z3], (-0.045302615503799264+0j) [Y0 Y1 X2 X3]],
 8: [(0.17141282644776892+0j) [Z1], (0.045302615503799264+0j) [Y0 X1 X2 Y3]],
 9: [(-0.22343153690813564+0j) [Z2], (-0.045302615503799264+0j) [X0 X1 Y2 Y3]],
 10: [(0.045302615503799264+0j) [X0 Y1 Y2 X3], (0.1714128264477689+0j) [Z0]]}

In [17]:
from qiskit.extensions import UnitaryGate
def Custom_gate_from_matrix(unitary_matrix, gate_label):
    
    return UnitaryGate(unitary_matrix, label='{}'.format(gate_label))


# convert everything to qiskit circuits!
- note indexing goes from right to left in qiskit!

https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.reverse_bits.html#qiskit.circuit.QuantumCircuit.reverse_bits

In [18]:
from qiskit.quantum_info import Operator
def Get_matrix_from_IBM_circuit(q_circuit):
    # https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.reverse_bits.html#qiskit.circuit.QuantumCircuit.reverse_bits
    reversed_bit_circuit = q_circuit.copy().reverse_bits()
    return Operator(reversed_bit_circuit).data

In [19]:
# checking by reversing bit order in Q circuit
op = QubitOperator('Z2', 1)
NQ=3

cirq_mat = get_sparse_operator(op, n_qubits=NQ).todense()
q_regggg = QuantumRegister(NQ)
ccc = QuantumCircuit(q_regggg)
ccc.z(q_regggg[2])
print(ccc.draw())

op = Operator(ccc)
IBM_mat = op.data
print(np.allclose(IBM_mat, cirq_mat))

# using function that reverses bit order of circuit!
print(np.allclose(Get_matrix_from_IBM_circuit(ccc), cirq_mat))

           
q0_0: ─────
           
q0_1: ─────
      ┌───┐
q0_2: ┤ Z ├
      └───┘
False
True


In [20]:
def HF_state_IBM_circuit(HF_state, q_register, q_circuit):
    ## re-order IN REVERSE!!!!!!!!!!!!!! IMPORTANT!!!!!

    for qNo, bit in enumerate(HF_state):
        if bit == 1:
            q_circuit.x(q_register[qNo])
        elif bit == 0:
            continue
        else:
            raise ValueError('HF state not binary: {}'.format(HF_state))

    return q_circuit

In [21]:
q_reg = QuantumRegister(len(new_input_state))
qcirc = QuantumCircuit(q_reg)
qcirc = HF_state_IBM_circuit(new_input_state, q_reg, qcirc)
qcirc.draw()

┌───┐
q1_0: ┤ X ├
      ├───┤
q1_1: ┤ X ├
      └───┘
q1_2: ─────
           
q1_3: ─────


From PHYS. REV. X, **8**, 031022 (2018):

$$U = e^{-i \theta Y_{0} X_{1} X_{2} X_{3}}$$

- when acting on $| 1100 \rangle_{HF-JW-STATE}$ 

$$U | \psi_{HF}\rangle = | \psi_{UCCSD}\rangle$$

to do this in Q.C the following circuit is employed:

In [22]:
from qiskit.extensions import UnitaryGate
def My_Rz_gate(theta):
    unitary_matrix = cirq.rz(theta)._unitary_()

    return UnitaryGate(unitary_matrix, label='My_Rz({})'.format(np.around(theta, 3)))

In [23]:
def exp_PauliWord_circuit_IBM(PauliWord, theta, q_register, q_circuit):
    q_circuit = q_circuit.copy()
    qubitNos, PauliStrs = zip(*list(*PauliWord.terms.keys()))

    control_qubit = max(qubitNos)
    min_qubit = min(qubitNos)

    # change basis
    for index, qNo in enumerate(qubitNos):
        Pstr = PauliStrs[index]
        qNo = int(qNo)
        if Pstr == 'X':
            q_circuit.h(q_register[qNo])
        elif Pstr == 'Y':
            q_circuit.rx((+np.pi / 2), q_register[qNo])
        elif Pstr == 'Z':
            continue
        else:
            raise ValueError('Not a PauliWord')

    # entangle
    for index, qNo in enumerate(qubitNos):
        Pstr = PauliStrs[index]
        qNo = int(qNo)
        if qNo < control_qubit:
            next_qubit = int(qubitNos[index + 1])
            q_circuit.cx(q_register[qNo], q_register[next_qubit])

    # rz
    for index, qNo in enumerate(qubitNos):
        Pstr = PauliStrs[index]
        qNo = int(qNo)
        if qNo == control_qubit:
            cofactor = list(PauliWord.terms.values())[0]

            if isinstance(cofactor, complex):
                if cofactor.imag < 0:
                    Rzgate = My_Rz_gate((2 * theta * np.abs(cofactor.imag)).real)
                    q_circuit.append(Rzgate, [control_qubit])
                #                     q_circuit.rz((2 * theta * np.abs(cofactor.imag)).real, q_register[control_qubit])
                else:
                    # times angle by negative one to get implementation
                    Rzgate = My_Rz_gate((2 * theta * np.abs(cofactor.imag) * -1).real)
                    q_circuit.append(Rzgate, [control_qubit])
            #                     q_circuit.rz((2 * theta *  np.abs(cofactor.imag) *-1).real, q_register[control_qubit])
            else:
                raise ValueError('PauliWord needs complex part to exponentiate')

    # entangle
    for index, qNo in enumerate(qubitNos[::-1]):
        qNo = int(qNo)
        if min_qubit < qNo:
            next_qubit = int(qubitNos[::-1][index + 1])
            q_circuit.cx(q_register[next_qubit], q_register[qNo])

        # undo basis change
    for index, qNo in enumerate(qubitNos):
        Pstr = PauliStrs[index]
        qNo = int(qNo)
        if Pstr == 'X':
            q_circuit.h(q_register[qNo])
        elif Pstr == 'Y':
            q_circuit.rx((-np.pi / 2), q_register[qNo])
        elif Pstr == 'Z':
            continue

    return q_circuit

In [24]:
ansatz_Op = QubitOperator('Y0 X1 X2 X3', -1j)
theta=3.27
circuit = exp_PauliWord_circuit_IBM(ansatz_Op, theta, q_reg, qcirc)
print(circuit.draw())

backend = Aer.get_backend('statevector_simulator')
job = execute(circuit, backend)
qc_state = job.result().get_statevector(circuit)
np.around(qc_state, 3)

      ┌───┐┌──────────┐                                             »
q1_0: ┤ X ├┤ RX(pi/2) ├──■───────────────────────────────────────■──»
      ├───┤└──┬───┬───┘┌─┴─┐                                   ┌─┴─┐»
q1_1: ┤ X ├───┤ H ├────┤ X ├──■─────────────────────────────■──┤ X ├»
      ├───┤   └───┘    └───┘┌─┴─┐                         ┌─┴─┐├───┤»
q1_2: ┤ H ├─────────────────┤ X ├──■───────────────────■──┤ X ├┤ H ├»
      ├───┤                 └───┘┌─┴─┐┌─────────────┐┌─┴─┐├───┤└───┘»
q1_3: ┤ H ├──────────────────────┤ X ├┤ My_Rz(6.54) ├┤ X ├┤ H ├─────»
      └───┘                      └───┘└─────────────┘└───┘└───┘     »
«      ┌───────────┐
«q1_0: ┤ RX(-pi/2) ├
«      └───┬───┬───┘
«q1_1: ────┤ H ├────
«          └───┘    
«q1_2: ─────────────
«                   
«q1_3: ─────────────
«                   


array([-0.   +0.j,  0.   -0.j,  0.   -0.j, -0.992+0.j,  0.   -0.j,
       -0.   +0.j, -0.   +0.j, -0.   -0.j,  0.   -0.j, -0.   -0.j,
       -0.   +0.j, -0.   -0.j,  0.128-0.j, -0.   -0.j, -0.   +0.j,
       -0.   +0.j])

In [27]:
######### Ansatz circuit
from quchem.Simulating_Quantum_Circuit import *
from quchem.Ansatz_Generator_Functions import *
def H2_ansatz(theta):
    HF_circ = [cirq.X.on(cirq.LineQubit(0)), cirq.X.on(cirq.LineQubit(1))]
    
    full_exp_circ_obj = full_exponentiated_PauliWord_circuit(QubitOperator('Y0 X1 X2 X3', -1j), theta)
    UCCSD_circ = cirq.Circuit(cirq.decompose_once((full_exp_circ_obj(*cirq.LineQubit.range(full_exp_circ_obj.num_qubits())))))
    full_circuit = cirq.Circuit([*HF_circ, *UCCSD_circ.all_operations()])
    
    return full_circuit
    
cirq_circuit = H2_ansatz(theta)  
print(cirq_circuit)

if Molecule=='H2' and transformation=='JW':
    matrix_IBM = Get_matrix_from_IBM_circuit(circuit)
    print('checking IBM and cirq circuits:', np.allclose(cirq_circuit.unitary(), matrix_IBM))

0: ───X───Rx(0.5π)───@──────────────────────────────────────────────@───Rx(-0.5π)───
                     │                                              │
1: ───X───H──────────X───@──────────────────────────────────────@───X───H───────────
                         │                                      │
2: ───H──────────────────X───@──────────────────────────────@───X───H───────────────
                             │                              │
3: ───H──────────────────────X───Rz(-1.9180000000000001π)───X───H───────────────────
checking IBM and cirq circuits: True


In [28]:
q_reg = QuantumRegister(len(new_input_state))
qcirc = QuantumCircuit(q_reg)
qcirc = HF_state_IBM_circuit(new_input_state, q_reg, qcirc)

ansatz_Op = QubitOperator('Y0 X1 X2 X3', 1j)
theta=0.12
perfect_ansatz_circ = exp_PauliWord_circuit_IBM(ansatz_Op, theta, q_reg, qcirc)
perfect_ansatz_circ.draw()

┌───┐┌──────────┐                                              »
q8_0: ┤ X ├┤ RX(pi/2) ├──■────────────────────────────────────────■──»
      ├───┤└──┬───┬───┘┌─┴─┐                                    ┌─┴─┐»
q8_1: ┤ X ├───┤ H ├────┤ X ├──■──────────────────────────────■──┤ X ├»
      ├───┤   └───┘    └───┘┌─┴─┐                          ┌─┴─┐├───┤»
q8_2: ┤ H ├─────────────────┤ X ├──■────────────────────■──┤ X ├┤ H ├»
      ├───┤                 └───┘┌─┴─┐┌──────────────┐┌─┴─┐├───┤└───┘»
q8_3: ┤ H ├──────────────────────┤ X ├┤ My_Rz(-0.24) ├┤ X ├┤ H ├─────»
      └───┘                      └───┘└──────────────┘└───┘└───┘     »
«      ┌───────────┐
«q8_0: ┤ RX(-pi/2) ├
«      └───┬───┬───┘
«q8_1: ────┤ H ├────
«          └───┘    
«q8_2: ─────────────
«                   
«q8_3: ─────────────
«

In [29]:
def change_basis_for_Z_measure(PauliWord, q_register, q_circuit):
    q_circuit = q_circuit.copy()

    qubitNos, PauliStrs = zip(*list(*PauliWord.terms.keys()))

    # change basis
    for index, qNo in enumerate(qubitNos):
        qNo = int(qNo)
        Pstr = PauliStrs[index]
        if Pstr == 'X':
            q_circuit.h(q_register[qNo])

        elif Pstr == 'Y':
            q_circuit.rx((+np.pi / 2), q_register[qNo])

        elif Pstr == 'Z':
            continue

        else:
            raise ValueError('Not a PauliWord')

    return q_circuit

In [ ]:
# # REVERSE BITS!
# def change_basis_for_Z_measure(PauliWord, q_register, q_circuit):
    
#     meas_circuit = QuantumCircuit(q_register)
    

#     qubitNos, PauliStrs = zip(*list(*PauliWord.terms.keys()))

#     # change basis
#     for index, qNo in enumerate(qubitNos):
#         qNo = int(qNo)
#         Pstr = PauliStrs[index]
#         if Pstr == 'X':
#             meas_circuit.h(q_register[qNo])

#         elif Pstr == 'Y':
#             meas_circuit.rx((+np.pi / 2), q_register[qNo])

#         elif Pstr == 'Z':
#             continue

#         else:
#             raise ValueError('Not a PauliWord')
            
#     meas_circuit = meas_circuit.reverse_bits()
    
#     return q_circuit.combine(meas_circuit)

In [30]:
def calc_exp_pauliword(count_dict, PauliWord):
    # takes correct part of bitstring when all lines measured

    qubitNos, PauliStrs = zip(*list(*PauliWord.terms.keys()))
    n_zeros = 0
    n_ones = 0

    for bitstring in count_dict:
        measure_term = np.take([int(bit) for bit in bitstring[::-1]], qubitNos) #reverse order here!

        parity_m_term = sum(measure_term) % 2

        if parity_m_term == 0:
            n_zeros += count_dict[bitstring]
        elif parity_m_term == 1:
            n_ones += count_dict[bitstring]
        else:
            raise ValueError('state {} not allowed'.format(measure_term))

    expectation_value = (n_zeros - n_ones) / (n_zeros + n_ones)

    return expectation_value

In [31]:
import qiskit

def arb_state_initalize_circuit(state_to_prepare, q_register, q_circuit):
    q_circuit=q_circuit.copy()
    state_to_prepare = np.asarray(state_to_prepare)
    q_circuit.initialize(state_to_prepare.tolist(), q_register)
    return q_circuit

def Get_Q_circ_to_build_state(arb_state,q_reg, qcirc, check_state=False):
    # https://qiskit.org/documentation/_modules/qiskit/extensions/quantum_initializer/initializer.html

    # assumes logical zero input state.
    # gives quantum circuit to prepare state (use decompose to get standard gates)
    # the qiskit.quantum_info.Operator function can be used to get the unitary matrix of the quantum circuit!

    qcirc = arb_state_initalize_circuit(arb_state, q_reg, qcirc)
    qcirc = qcirc.decompose()

    # need to remove reset part of circuit
    new_data = []
    for index, tup in enumerate(qcirc.data):
        op_type, _, _ = tup
        if isinstance(op_type, qiskit.circuit.reset.Reset):
            continue
        else:
            new_data.append(tup)
    qcirc.data = new_data

    if check_state:
        backend = Aer.get_backend('statevector_simulator')
        job = execute(qcirc, backend)
        qc_state = job.result().get_statevector(qcirc)

        if not np.allclose(qc_state, arb_state):
            raise ValueError('Incorrect state being prepared')

    return qcirc

In [32]:
n_qubits = int(np.log2(len(ground_state_vector)))
backend=Aer.get_backend('qasm_simulator')
# n_shots=1024
n_shots=100_000

q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)
perfect_ansatz_circ=Get_Q_circ_to_build_state(ground_state_vector,
                                              q_reg,
                                              qcirc,
                                              check_state=False)  

perfect_ansatz_circ = perfect_ansatz_circ.reverse_bits() #reverse order here!

E_list=[]
# for qubitOp in NewQubitHamiltonian_relabelled:
for qubitOp in tqdm(NewQubitHamiltonian_relabelled, ascii=True, desc='performing VQE'):
    for PauliWord, const in qubitOp.terms.items():
        if PauliWord:
            full_circuit = change_basis_for_Z_measure(qubitOp,
                                                      q_reg,
                                                      perfect_ansatz_circ,
                                                      )
            full_circuit.measure_all()
            
#             print(qubitOp)
#             print(full_circuit.draw())
            
            job = execute(full_circuit, backend, shots=n_shots)
            result = job.result()
            exp_counts_dict = result.get_counts(full_circuit)
            
            exp_val = calc_exp_pauliword(exp_counts_dict, qubitOp)
            E_list.append(exp_val*const)
        else:
            E_list.append(const)

print('FCI expected =', new_FCI_Energy)    
print('measured E = ', sum(E_list))
print('success:', np.allclose(sum(E_list), new_FCI_Energy))


FCI expected = (-1.137283834488503+3.7121881080805557e-17j)
measured E =  (-1.1368302156445558+0j)
success: False


In [33]:
from qiskit.circuit.library.standard_gates import XGate, YGate, ZGate
def IBM_PauliWord(PauliOp, N_qubits, draw=False, reverse=False):
    
    
    qubitNos, PauliStrs = zip(*list(*PauliOp.terms.keys()))
    
#     # re-order IN REVERSE!!!!!!!!!!!!!! IMPORTANT!!!!!
#     relabel_dic = {index: qubitNo for index, qubitNo in enumerate(range(N_qubits)[::-1])}
#     qubitNos = [relabel_dic[qNo] for qNo in qubitNos]
#     idx = np.argsort(qubitNos)
#     qubitNos = np.array(qubitNos)[idx].tolist()
#     PauliStrs = np.array(PauliStrs)[idx].tolist()
#     ## !!!!!!!!!!!!! IMPORTANT!!!!!
    
    q_register = QuantumRegister(N_qubits)
    q_circuit = QuantumCircuit(q_register)
    
    for qNo in range(N_qubits):
        if qNo in  qubitNos:
            index=qubitNos.index(qNo)
            Pstr = PauliStrs[index]
            if Pstr == 'X':
                q_circuit.x(q_register[qNo])
            elif Pstr == 'Y':
                q_circuit.y(q_register[qNo])
            elif Pstr == 'Z':
                q_circuit.z(q_register[qNo])
            else:
                raise ValueError('Not a Pauli {}'.format(Pstr))
        else:
            q_circuit.i(q_register[qNo])
    
    
    
    if reverse:
        q_circuit=q_circuit.reverse_bits()
        if draw:
            print(q_circuit.draw())
        return Operator(q_circuit).data
    else:
        if draw:
            print(q_circuit.draw())
        return Operator(q_circuit).data
IBM_PauliWord(QubitOperator('Y0 X2', 1),3, draw=True, reverse=False)

        ┌───┐
q141_0: ┤ Y ├
        ├───┤
q141_1: ┤ I ├
        ├───┤
q141_2: ┤ X ├
        └───┘


array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j],
       [0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [34]:
# LINEAR ALG METHOD!

q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)
perfect_ansatz_circ=Get_Q_circ_to_build_state(ground_state_vector,
                                              q_reg,
                                              qcirc,
                                              check_state=False) 

perfect_ansatz_circ = perfect_ansatz_circ.reverse_bits() #reverse order here!

backend = Aer.get_backend('statevector_simulator')
job = execute(perfect_ansatz_circ, backend)
ANSATZ_STATE = job.result().get_statevector(perfect_ansatz_circ)
ANSATZ_bra = ANSATZ_STATE.conj().T


E_list=[]
# for qubitOp in NewQubitHamiltonian_relabelled:
for qubitOp in tqdm(NewQubitHamiltonian_relabelled, ascii=True, desc='performing VQE'):
    for PauliWord, const in qubitOp.terms.items():
        if PauliWord:
            Pauli_matrix = IBM_PauliWord(qubitOp, n_qubits, draw=False)

            exp_val = np.dot(ANSATZ_bra, Pauli_matrix.dot(ANSATZ_STATE))
            E_list.append(exp_val*const)
        else:
            E_list.append(const)

print('FCI expected =', new_FCI_Energy)    
print('measured E = ', sum(E_list))
print('success:', np.allclose(sum(E_list), new_FCI_Energy))


FCI expected = (-1.137283834488503+3.7121881080805557e-17j)
measured E =  (-1.1372838344885017+0j)
success: True


In [35]:
def IBM_gate_from_matrix(unitary_matrix, q_circuit, gate_str=None):
    
    if isinstance(unitary_matrix, scipy.sparse.csc.csc_matrix):
        unitary_matrix=unitary_matrix.toarray()
    
    
    q_circuit=q_circuit.copy()
    N_qubits = q_circuit.num_qubits
    
    if unitary_matrix.shape != (2**N_qubits, 2**N_qubits):
        raise ValueError('defined matrix is incorrect dimention')
    
    qubit_list = [i for i in range(N_qubits)]
    custom_gate = UnitaryGate(unitary_matrix, label='Custom Gate' if gate_str is None else '{}'.format(gate_str))
    
    del unitary_matrix
    q_circuit.append(custom_gate, qubit_list)
    return q_circuit

In [ ]:
# from quchem.Unitary_partitioning_Seq_Rot import *
# # REVERSE BITS!
# def Build_reduction_circuit_seq_rot_IBM(anti_commuting_set, S_index, q_circuit,q_register, n_qubits, check_reduction=False):
#     """
#     Function to build R_S (make up of all R_SK terms)

#     Args:
#         anti_commuting_set(list): list of anti commuting QubitOperators
#         S_index(int): index for Ps in anti_commuting_set list
#         check_reduction (optional, bool): use linear algebra to check that 𝑅s† 𝐻s 𝑅s == 𝑃s
#     returns:
#         full_RS_circuit(cirq.Circuit): Q_circuit for R_s operator
#         Ps (QubitOperator): Pauli_S operator with cofactor of 1!
#         gamma_l (float): normalization term

#     """
#     q_circuit = q_circuit.copy()
#     X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(anti_commuting_set, S_index)
    
#     seq_R_circuit= QuantumCircuit(q_register)
#     for X_sk_Op, theta_sk in X_sk_theta_sk_list:

#         pauliword_X_sk = list(X_sk_Op.terms.keys())[0]
#         const_X_sk = list(X_sk_Op.terms.values())[0]

#         seq_R_circuit = exp_PauliWord_circuit_IBM(QubitOperator(pauliword_X_sk, -1j), theta_sk/2*const_X_sk, q_register, seq_R_circuit)
    
    
    
#     seq_R_circuit=seq_R_circuit.reverse_bits() #reverse order here! (fully built first!)
#     if check_reduction:

#         H_S=QubitOperator()
#         for op in full_normalised_set['PauliWords']:
#             H_S+=op
#         H_S_matrix=get_sparse_operator(H_S, n_qubits=n_qubits)
        
#         Ps_mat = get_sparse_operator(Ps, n_qubits=n_qubits)
#         R_S_matrix= Operator(seq_R_circuit).data
        
#         reduction_mat = R_S_matrix.dot(H_S_matrix.dot(R_S_matrix.conj().transpose()))
        
#         if not (np.allclose(Ps_mat.todense(), reduction_mat)):
# #         if not (np.allclose(Ps_mat.todense(), reduction_mat.todense())):
#             print('reduction circuit incorrect...   𝑅s 𝐻s 𝑅s† != 𝑃s') 
# #     return q_circuit.combine(seq_R_circuit), Ps, gamma_l
#     return seq_R_circuit, Ps, gamma_l

In [36]:
from quchem.Unitary_partitioning_Seq_Rot import *

def Build_reduction_circuit_seq_rot_IBM(anti_commuting_set, S_index, q_circuit,q_register, n_qubits, check_reduction=False):
    """
    Function to build R_S (make up of all R_SK terms)

    Args:
        anti_commuting_set(list): list of anti commuting QubitOperators
        S_index(int): index for Ps in anti_commuting_set list
        check_reduction (optional, bool): use linear algebra to check that 𝑅s† 𝐻s 𝑅s == 𝑃s
    returns:
        full_RS_circuit(cirq.Circuit): Q_circuit for R_s operator
        Ps (QubitOperator): Pauli_S operator with cofactor of 1!
        gamma_l (float): normalization term

    """
    q_circuit = q_circuit.copy()
    X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(anti_commuting_set, S_index)
    
    seq_R_circuit= QuantumCircuit(q_register)
    for X_sk_Op, theta_sk in X_sk_theta_sk_list:

        pauliword_X_sk = list(X_sk_Op.terms.keys())[0]
        const_X_sk = list(X_sk_Op.terms.values())[0]

        seq_R_circuit = exp_PauliWord_circuit_IBM(QubitOperator(pauliword_X_sk, -1j), theta_sk/2*const_X_sk, q_register, seq_R_circuit)
    
    if check_reduction:

        H_S=QubitOperator()
        for op in full_normalised_set['PauliWords']:
            H_S+=op
        H_S_matrix=get_sparse_operator(H_S, n_qubits=n_qubits)
        
        Ps_mat = get_sparse_operator(Ps, n_qubits=n_qubits)
        R_S_matrix= Get_matrix_from_IBM_circuit(seq_R_circuit)
        
        reduction_mat = R_S_matrix.dot(H_S_matrix.dot(R_S_matrix.conj().transpose()))
        
        if not (np.allclose(Ps_mat.todense(), reduction_mat)):
#         if not (np.allclose(Ps_mat.todense(), reduction_mat.todense())):
            print('reduction circuit incorrect...   𝑅s 𝐻s 𝑅s† != 𝑃s') 
#     return q_circuit.combine(seq_R_circuit), Ps, gamma_l
    return seq_R_circuit, Ps, gamma_l

In [37]:
q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)
perfect_ansatz_circ=Get_Q_circ_to_build_state(ground_state_vector,
                                              q_reg,
                                              qcirc,
                                              check_state=False)  #reverse order here!

S_index=0
set_key=3

X_sk_theta_sk_list, full_normalised_set, Ps, gamma_l = Get_Xsk_op_list(anti_commuting_sets[set_key], S_index)
print(X_sk_theta_sk_list)
# print(Ps)

xx, _, _=Build_reduction_circuit_seq_rot_IBM(anti_commuting_sets[set_key],
                                    S_index,
                                    perfect_ansatz_circ,
                                    q_reg,
                                    n_qubits,      
                                    check_reduction=True)
xx.draw()

[]


q213_0: 
        
q213_1: 
        
q213_2: 
        
q213_3:

In [38]:
## cirq check

ansatz_cirq_circuit = cirq.Circuit([cirq.I.on(i) for i in cirq.LineQubit.range(n_qubits)])

full_circuit, Ps, gamma_l = Generate_Full_Q_Circuit_Conj_NO_M_gates(ansatz_cirq_circuit, 
                                                         anti_commuting_sets[set_key], 
                                                         S_index, 
                                                         check_reduction=True)
full_circuit

0: ───I───

1: ───I───

2: ───I───

3: ───I───

In [39]:
np.allclose(full_circuit.unitary(), Operator(xx.reverse_bits()).data)

True

In [40]:
Seq_Rot_circuits=[]
N_qubits = len(new_input_state)
S_index=0
check=True

backend=Aer.get_backend('qasm_simulator')
n_shots=10_000

q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)

perfect_ansatz_circ=Get_Q_circ_to_build_state(ground_state_vector,
                                              q_reg,
                                              qcirc,
                                              check_state=False) 

perfect_ansatz_circ = perfect_ansatz_circ.reverse_bits() #reverse order here!

E_list=[]
for set_key in tqdm(list(anti_commuting_sets.keys()), ascii=True, desc='performing VQE'):
    anti_set_list= anti_commuting_sets[set_key]
    if len(anti_set_list)>1:
        
        R_sl_circuit, Ps, gamma_l = Build_reduction_circuit_seq_rot_IBM(
                                    anti_set_list,
                                    S_index,
                                    perfect_ansatz_circ,
                                    q_reg,
                                    N_qubits,      
                                    check_reduction=check)
        
        
#         R_sl_circuit = R_sl_circuit.reverse_bits() # reverse Rsl
        combined_circuits = perfect_ansatz_circ.combine(R_sl_circuit)
        
        full_circuit = change_basis_for_Z_measure(Ps, 
                                                  q_reg,
                                                  combined_circuits)
        full_circuit.measure_all()
        
#         print(Ps)
#         print(full_circuit.draw())

        job = execute(full_circuit, backend, shots=n_shots)
        result = job.result()
        exp_counts_dict = result.get_counts(full_circuit)
        
        exp_val = calc_exp_pauliword(exp_counts_dict, Ps)
        E_list.append(exp_val*gamma_l)
        
    else:
        qubitOp = anti_set_list[0]
        for PauliWord, const in qubitOp.terms.items():
            if PauliWord:
                full_circuit = change_basis_for_Z_measure(qubitOp, 
                                                          q_reg,
                                                          perfect_ansatz_circ)
                full_circuit.measure_all()

                job = execute(full_circuit, backend, shots=n_shots)
                result = job.result()
                exp_counts_dict = result.get_counts(full_circuit)

                exp_val = calc_exp_pauliword(exp_counts_dict, qubitOp)
                E_list.append(exp_val*const)
            else:
                E_list.append(const)
                
print('FCI expected =', new_FCI_Energy)    
print('measured E = ', sum(E_list))
print('success:', np.allclose(sum(E_list), new_FCI_Energy))


FCI expected = (-1.137283834488503+3.7121881080805557e-17j)
measured E =  (-1.137238203171067+0j)
success: False


In [41]:
# lin alg method!

Seq_Rot_circuits=[]
N_qubits = len(new_input_state)
S_index=0
check=True

backend=Aer.get_backend('qasm_simulator')
n_shots=10_000

q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)

perfect_ansatz_circ=Get_Q_circ_to_build_state(ground_state_vector,
                                              q_reg,
                                              qcirc,
                                              check_state=False) 

perfect_ansatz_circ = perfect_ansatz_circ.reverse_bits() #reverse order here!

backend = Aer.get_backend('statevector_simulator')
job = execute(perfect_ansatz_circ, backend)
ANSATZ_STATE = job.result().get_statevector(perfect_ansatz_circ)
ANSATZ_bra = ANSATZ_STATE.conj().T


E_list=[]
for set_key in tqdm(list(anti_commuting_sets.keys()), ascii=True, desc='performing VQE'):
    anti_set_list= anti_commuting_sets[set_key]
    if len(anti_set_list)>1:
        
        R_sl_circuit, Ps, gamma_l = Build_reduction_circuit_seq_rot_IBM(
                                    anti_set_list,
                                    S_index,
                                    perfect_ansatz_circ,
                                    q_reg,
                                    N_qubits,      
                                    check_reduction=check)
        
        
#         R_sl_circuit=R_sl_circuit.reverse_bits()
        R_sl = Operator(R_sl_circuit).data #may need to reverse bits!
        Pauli_matrix = IBM_PauliWord(Ps, n_qubits, draw=False, reverse=False)
        
        post_Rsl_state = R_sl.dot(ANSATZ_STATE)
        
        exp_val = np.dot(post_Rsl_state.conj().T, Pauli_matrix.dot(post_Rsl_state))
        E_list.append(exp_val*gamma_l)
        
    else:
        qubitOp = anti_set_list[0]
        for PauliWord, const in qubitOp.terms.items():
            if PauliWord:
                Pauli_matrix = IBM_PauliWord(qubitOp, n_qubits, draw=False, reverse=False)
                exp_val = np.dot(ANSATZ_bra, Pauli_matrix.dot(ANSATZ_STATE))
                E_list.append(exp_val*const)
            else:
                E_list.append(const)

print('FCI expected =', new_FCI_Energy)    
print('measured E = ', sum(E_list))
print('success:', np.allclose(sum(E_list), new_FCI_Energy))


FCI expected = (-1.137283834488503+3.7121881080805557e-17j)
measured E =  (-1.137283834488501+0j)
success: True


# LCU

In [42]:
from quchem.Unitary_partitioning_LCU_method import *

In [ ]:

No_control_qubits=1
No_system_qubits=4

R_corrected_Op_list = [QubitOperator('', 0.3020015943219484), QubitOperator('X0 Y1 X2 X3', 0.953307419999971)]
R_corr_list = [1, -1j]
Pn = QubitOperator('Y1', 1)



R_circ_obj = LCU_R_gate(No_control_qubits, No_system_qubits, R_corrected_Op_list, R_corr_list, Pn)
R_circ_circ = cirq.Circuit(
cirq.decompose_once((R_circ_obj(*cirq.LineQubit.range(R_circ_obj.num_qubits())))))
R_circ_circ

In [43]:
def phase_Pauli_gate(Pstr, cofactor):
    if Pstr == 'X':
        unitary_matrix = cofactor * np.array([[0, 1], [1, 0]], dtype=complex)
    elif Pstr == 'Y':
        unitary_matrix = cofactor * np.array([[0, -1j], [1j, 0]], dtype=complex)
    elif Pstr == 'Z':
        unitary_matrix = cofactor * np.array([[1, 0], [0, -1]], dtype=complex)
    else:
        raise ValueError('P_str is not a Pauli')

    return UnitaryGate(unitary_matrix, label='{}*{}'.format(cofactor, Pstr))

In [44]:
from qiskit.circuit.library.standard_gates import XGate, YGate, ZGate
def control_P_IBM(PauliOp, phase_correction, control_index, q_circuit, n_qubits, n_ancilla, list_measured_qubits=None):
    
    q_circuit = q_circuit.copy()
    qubitNos, PauliStrs = zip(*list(*PauliOp.terms.keys()))
    
    control_indices_list = list(range(n_qubits, n_qubits+n_ancilla))
    
    if list_measured_qubits is None:
        qubit_to_put_phase_on=0
    else:
        qubit_to_put_phase_on = list(set(qubitNos) & set(list_measured_qubits))[0]
        
    for index, qNo in enumerate(qubitNos):
        qNo = int(qNo)
        Pstr = PauliStrs[index]
        
        if qNo==qubit_to_put_phase_on:
            phase_P = phase_Pauli_gate(Pstr, phase_correction).control(n_ancilla)
            phase_P.num_ctrl_qubits =n_ancilla
            phase_P.ctrl_state= control_index
#             q_circuit.append(phase_P, [*[i for i in range(0, n_ancilla)],qNo])
            q_circuit.append(phase_P, [*control_indices_list,qNo])
        else:
            if Pstr == 'X':
                X_gate = XGate().control(n_ancilla)
                X_gate.ctrl_state= control_index
                q_circuit.append(X_gate, [*control_indices_list,qNo])
#                 q_circuit.append(X_gate, [*[i for i in range(0, n_ancilla)],qNo])
            elif Pstr == 'Y':
                Y_gate = YGate().control(n_ancilla)
                Y_gate.ctrl_state= control_index
                q_circuit.append(Y_gate, [*control_indices_list,qNo])
#                 q_circuit.append(Y_gate, [*[i for i in range(0, n_ancilla)],qNo])
            elif Pstr == 'Z':
                Z_gate = ZGate().control(n_ancilla)
                Z_gate.ctrl_state= control_index
                q_circuit.append(Z_gate, [*control_indices_list,qNo])
#                 q_circuit.append(Z_gate, [*[i for i in range(0, n_ancilla)],qNo])
            
    return q_circuit

In [ ]:
q_reg = QuantumRegister(No_control_qubits+No_system_qubits)
qcirc = QuantumCircuit(q_reg)
control_index=1
list_qubits_to_measure, _ = zip(*list(*Pn.terms.keys()))
print(list_qubits_to_measure)

P = QubitOperator('X0 Y1 X2 X3', 1)
phase = -1j

xx =control_P_IBM(P, phase, control_index, qcirc, No_system_qubits, No_control_qubits, list_measured_qubits=list_qubits_to_measure)
xx.draw()

In [ ]:
np.allclose(R_circ_circ.unitary(), Operator(xx.reverse_bits()).data)

In [ ]:
# ancilla_amplitudes= [np.sqrt(0.4), np.sqrt(0.3), np.sqrt(0.2), np.sqrt(0.1)]
# ancilla_amplitudes= [np.sqrt(0.6), np.sqrt(0.4)]
ancilla_amplitudes= [np.sqrt(0.1), np.sqrt(0.8), np.sqrt(0.1), 0]

N_system_qubits=2

ancilla_obj = prepare_arb_state(ancilla_amplitudes, N_system_qubits)
ancilla_circ = ancilla_obj.Get_state_prep_Circuit()
ancilla_circ

In [ ]:
ancilla_circ.unitary()

In [ ]:
q_reg_ancilla = QuantumRegister(np.log2(len(ancilla_amplitudes)))
q_circ_ancilla = QuantumCircuit(q_reg_ancilla)

G_circuit = Get_Q_circ_to_build_state(ancilla_amplitudes, q_reg_ancilla, q_circ_ancilla, check_state=True)
print(G_circuit.draw())

Get_matrix_from_IBM_circuit(G_circuit) # only require first column to be equal!

In [ ]:
# note do not use Get_matrix_from_IBM_circuit here... Get_Q_circ_to_build_state doesn't need to be reversed!
print('checking_first_row_equal =', np.allclose(Operator(G_circuit).data[:,0], ancilla_circ.unitary()[:,0]))

In [ ]:
# backend = Aer.get_backend('statevector_simulator')
# job = execute(G_circuit, backend)
# qc_state = job.result().get_statevector(G_circuit)
# np.allclose(qc_state, ancilla_amplitudes)

In [ ]:
new_G=G_circuit.copy()
new_G.measure_all()
backend = Aer.get_backend('qasm_simulator')
job = execute(new_G, backend, shots=1024)
result = job.result()
exp_counts_dict = result.get_counts(new_G)
exp_counts_dict

In [ ]:
meas_obj = Measure_PauliWord(QubitOperator('Z2 Z3', 1))
meas_circ = cirq.Circuit( ancilla_circ, 
cirq.decompose_once((meas_obj(*cirq.LineQubit.range(meas_obj.num_qubits())))))
print(meas_circ)

simulator = cirq.Simulator()
raw_result = simulator.run(meas_circ, repetitions=1024)
histogram_string=Get_Histogram_key(QubitOperator('Z2 Z3', 1))
hist_result = raw_result.histogram(key=histogram_string)
Return_as_binary(hist_result, histogram_string)

In [ ]:
ancilla_amplitudes

In [45]:
# LCU_circuits=[]
N_qubits = len(new_input_state)
N_index=0
set_index=9

print(anti_commuting_sets[set_index])

backend=Aer.get_backend('qasm_simulator')
n_shots=1024

q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)
perfect_ansatz_circ=Get_Q_circ_to_build_state(ground_state_vector,
                                              q_reg,
                                              qcirc,
                                              check_state=False) 

perfect_ansatz_circ = perfect_ansatz_circ.reverse_bits() #reverse order here!




R_uncorrected, Pn, gamma_l = Get_R_op_list(anti_commuting_sets[set_index], N_index)
R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)



N_ancilla = int(np.ceil(np.log2(len(ancilla_amplitudes))))
if len(ancilla_amplitudes)!= 2**N_ancilla:
    n_missing = int(2**N_ancilla-len(ancilla_amplitudes))
    missing_terms = [0 for _ in range(n_missing)]
    ancilla_amplitudes = [*ancilla_amplitudes, *missing_terms]

q_reg_ancilla = QuantumRegister(N_ancilla)
q_circ_ancilla = QuantumCircuit(q_reg_ancilla)
G_circuit = Get_Q_circ_to_build_state(ancilla_amplitudes, q_reg_ancilla, q_circ_ancilla)
G_inverse=G_circuit.inverse()

combined_circuits =perfect_ansatz_circ.combine(G_circuit)



list_qubits_to_measure, _ = zip(*list(*Pn.terms.keys()))

control_index=1
combined_circuits =control_P_IBM(R_corrected_Op_list[1],
                  R_corr_list[1],
                  control_index,
                  combined_circuits, 
                  n_qubits, 
                  N_ancilla,
                  list_measured_qubits=list_qubits_to_measure)


# R_circ_obj = LCU_R_gate(No_control_qubits, No_system_qubits, R_corrected_Op_list, R_corr_list, Pn)
# R_circ_circ = cirq.Circuit(
# cirq.decompose_once((R_circ_obj(*cirq.LineQubit.range(R_circ_obj.num_qubits())))))
# print(R_circ_circ)

combined_circuits=combined_circuits.combine(G_inverse)

print(R_corrected_Op_list[1])

full_circuit = change_basis_for_Z_measure(Pn, 
                                          q_reg,
                                          combined_circuits)
print(Ps)
full_circuit.draw()

[(-0.22343153690813564+0j) [Z2], (-0.045302615503799264+0j) [X0 X1 Y2 Y3]]
0.953307419999971 [X0 X1 X2 Y3]
1 [X0 Y1 Y2 X3]


┌──────────────────┐┌───┐                                        
q450_0: ┤3                 ├┤ X ├────────────────────────────────────────
        │                  │└─┬─┘┌───┐                                   
q450_1: ┤2                 ├──┼──┤ X ├───────────────────────────────────
        │  disentangler_dg │  │  └─┬─┘┌──────┐                           
q450_2: ┤1                 ├──┼────┼──┤ 1j*X ├───────────────────────────
        │                  │  │    │  └──┬───┘┌───┐                      
q450_3: ┤0                 ├──┼────┼─────┼────┤ Y ├──────────────────────
        ├─────────────────┬┘  │    │     │    └─┬─┘┌────────────────────┐
q504_0: ┤ disentangler_dg ├───■────■─────■──────■──┤ disentangler_dg_dg ├
        └─────────────────┘                        └────────────────────┘

In [46]:
def Get_post_selection_counts_LCU(list_of_measurements, N_ancilla):
    # checks all zero state on ancilla line

    new_counts={}
    for binary_result_str in list_of_measurements:
        ancilla_state = int(binary_result_str[:N_ancilla],2)
        if ancilla_state==0:
            post_select_m_binary = binary_result_str[N_ancilla:]
            if post_select_m_binary in new_counts.keys():
                new_counts[post_select_m_binary]+=1
            else:
                new_counts[post_select_m_binary]=1
            
        else:
            continue

    return new_counts

In [47]:
Get_post_selection_counts_LCU(['0000', '0001', '1100', '0111'], 2)

{'00': 1, '01': 1}

In [54]:
# LCU_circuits=[]
N_qubits = len(new_input_state)
N_index=0

backend=Aer.get_backend('qasm_simulator')
n_shots=100_000

q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)

perfect_ansatz_circ=Get_Q_circ_to_build_state(ground_state_vector,
                                              q_reg,
                                              qcirc,
                                              check_state=False) 

perfect_ansatz_circ = perfect_ansatz_circ.reverse_bits() #reverse order here!


N_SYSTEM=int(np.log2(len(ground_state_vector)))
E_list=[]
for set_key in tqdm(list(anti_commuting_sets.keys()), ascii=True, desc='performing VQE'):
    anti_set_list= anti_commuting_sets[set_key]
    if len(anti_set_list)>1:

        R_uncorrected, Pn, gamma_l = Get_R_op_list(anti_set_list, N_index)
        R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)     
              
        N_ancilla = int(np.ceil(np.log2(len(ancilla_amplitudes))))
        if len(ancilla_amplitudes)!= 2**N_ancilla:
            n_missing = int(2**N_ancilla-len(ancilla_amplitudes))
            missing_terms = [0 for _ in range(n_missing)]
            ancilla_amplitudes = [*ancilla_amplitudes, *missing_terms]
        
        q_reg_ancilla = QuantumRegister(N_ancilla)
        q_circ_ancilla = QuantumCircuit(q_reg_ancilla)
        G_circuit = Get_Q_circ_to_build_state(ancilla_amplitudes, q_reg_ancilla, q_circ_ancilla)
        G_inverse=G_circuit.inverse()
        
        # combine ancilla and system
        combined_circuits =perfect_ansatz_circ.combine(G_circuit)
               
        
        # find qubits that are measured!
        Pn_qubitNos, _ = zip(*list(*Pn.terms.keys()))
       
        
        for control_index, op in enumerate(R_corrected_Op_list):
            phase_corr=R_corr_list[control_index]
            for PauliW, Const in op.terms.items():
                if PauliW:
                    combined_circuits =control_P_IBM(op,
                                              phase_corr,
                                              control_index,
                                              combined_circuits, 
                                              N_SYSTEM, 
                                              N_ancilla,
                                              list_measured_qubits=Pn_qubitNos)
                else:
                    continue
        
        # G dag
        combined_circuits=combined_circuits.combine(G_inverse)
        
        full_circuit = change_basis_for_Z_measure(Pn, 
                                                  q_reg,
                                                  combined_circuits)
        full_circuit.measure_all()
        
        
        job = execute(full_circuit, backend, shots=n_shots, memory=True) # need memory for post_selection!
        result = job.result()
        list_of_results=result.get_memory(full_circuit)
        
        post_selected_dict = Get_post_selection_counts_LCU(list_of_results, N_ancilla)
        
        exp_val = calc_exp_pauliword(post_selected_dict, Pn)
        E_list.append(exp_val*gamma_l)
        
    else:
        qubitOp = anti_set_list[0]
        for PauliWord, const in qubitOp.terms.items():
            if PauliWord:
                full_circuit = change_basis_for_Z_measure(qubitOp, 
                                                          q_reg,
                                                          perfect_ansatz_circ)
                full_circuit.measure_all()

                job = execute(full_circuit, backend, shots=n_shots)
                result = job.result()
                exp_counts_dict = result.get_counts(full_circuit)

                exp_val = calc_exp_pauliword(exp_counts_dict, qubitOp)
                E_list.append(exp_val*const)
            else:
                E_list.append(const)

print('FCI expected =', new_FCI_Energy)    
print('measured E = ', sum(E_list))
print('success:', np.allclose(sum(E_list), new_FCI_Energy))


FCI expected = (-1.137283834488503+3.7121881080805557e-17j)
measured E =  (-1.1372680141615952+0j)
success: False


In [50]:
def POVM_LCU(n_system_q, n_ancilla_q, system_ancilla_output_ket):
    # state_vector_simulator the state is given as (ancilla X_kron system)
    
    full_density_matrix = np.outer(system_ancilla_output_ket, system_ancilla_output_ket)
    
    I_system_operator = np.eye((2**n_system_q))
    
    ancilla_0_state = np.eye(2**n_ancilla_q)[0,:]
    ancilla_0_projector = np.outer(ancilla_0_state, ancilla_0_state)
    
    POVM_0_ancilla = np.kron(ancilla_0_projector, I_system_operator)
    Kraus_Op_0 = POVM_0_ancilla.copy()
    
    term = Kraus_Op_0.dot(full_density_matrix.dot(Kraus_Op_0.transpose().conj()))
    projected_density_matrix = term/np.trace(term) # projected into correct space using POVM ancilla measurement!
    
#     ## Next get partial density matrix over system qubits # aka partial trace!
#     # https://scicomp.stackexchange.com/questions/27496/calculating-partial-trace-of-array-in-numpy
#     # reshape to do the partial trace easily using np.einsum
    
#     reshaped_dm = projected_density_matrix.reshape([2 ** n_system_q, 2 ** n_ancilla_q,
#                                                     2 ** n_system_q, 2 ** n_ancilla_q])
#     reduced_dm = np.einsum('jiki->jk', reshaped_dm)
    
    
    # p_a = sum_{b} (I_{a}*<b|) p_{ab} (I_{a}*|b>)
    basis_ancilla = np.eye((2 ** n_ancilla_q))
    reduced_dm = np.zeros((2 ** n_system_q, 2 ** n_system_q), dtype=complex)
    for b in range(basis_ancilla.shape[0]):
        b_ket = basis_ancilla[b, :].reshape([2 ** n_ancilla_q, 1])
        I_a_b_ket = np.kron(b_ket, I_system_operator)
#         I_a_b_ket = np.kron(I_system_operator, b_ket)
        I_a_b_bra = I_a_b_ket.transpose().conj()

        term = I_a_b_bra.dot(projected_density_matrix.dot(I_a_b_ket))
        reduced_dm += term
    
    
    return reduced_dm

In [55]:
# LCU_circuits=[]
N_qubits = len(new_input_state)
N_index=0

backend=Aer.get_backend('qasm_simulator')
n_shots=10_000

q_reg = QuantumRegister(n_qubits)
qcirc = QuantumCircuit(q_reg)

perfect_ansatz_circ=Get_Q_circ_to_build_state(ground_state_vector,
                                              q_reg,
                                              qcirc,
                                              check_state=False) 

perfect_ansatz_circ = perfect_ansatz_circ.reverse_bits() #reverse order here!


backend = Aer.get_backend('statevector_simulator')
job = execute(perfect_ansatz_circ, backend)
ANSATZ_STATE = job.result().get_statevector(perfect_ansatz_circ)
ANSATZ_bra = ANSATZ_STATE.conj().T


E_list=[]
for set_key in tqdm(list(anti_commuting_sets.keys()), ascii=True, desc='performing VQE'):
    anti_set_list= anti_commuting_sets[set_key]
    if len(anti_set_list)>1:

        R_uncorrected, Pn, gamma_l = Get_R_op_list(anti_set_list, N_index)
        R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)     
              
        N_ancilla = int(np.ceil(np.log2(len(ancilla_amplitudes))))
        if len(ancilla_amplitudes)!= 2**N_ancilla:
            n_missing = int(2**N_ancilla-len(ancilla_amplitudes))
            missing_terms = [0 for _ in range(n_missing)]
            ancilla_amplitudes = [*ancilla_amplitudes, *missing_terms]
        
        q_reg_ancilla = QuantumRegister(N_ancilla)
        q_circ_ancilla = QuantumCircuit(q_reg_ancilla)
        G_circuit = Get_Q_circ_to_build_state(ancilla_amplitudes, q_reg_ancilla, q_circ_ancilla)
        G_inverse=G_circuit.inverse()
        
        # combine ancilla and system
        combined_circuits =perfect_ansatz_circ.combine(G_circuit)
               
        
        # find qubits that are measured!
        Pn_qubitNos, _ = zip(*list(*Pn.terms.keys()))
       
        
        for control_index, op in enumerate(R_corrected_Op_list):
            phase_corr=R_corr_list[control_index]
            for PauliW, Const in op.terms.items():
                if PauliW:
                    combined_circuits =control_P_IBM(op,
                                              phase_corr,
                                              control_index,
                                              combined_circuits, 
                                              N_SYSTEM, 
                                              N_ancilla,
                                              list_measured_qubits=Pn_qubitNos)
                else:
                    continue
        
        # G dag
        combined_circuits=combined_circuits.combine(G_inverse)
#         combined_circuits=combined_circuits.reverse_bits()
        
       
#         print(combined_circuits.draw())
        job = execute(combined_circuits, backend)
        ANSATZ_and_ANCILLA = job.result().get_statevector(combined_circuits)
       
        
        partial_density_matrix = POVM_LCU(n_qubits, N_ancilla, ANSATZ_and_ANCILLA)
        
        Pn_system_only = IBM_PauliWord(Pn, n_qubits, draw=False, reverse=False)
#         print(Pn_system_only.shape)
#         print(partial_density_matrix.shape)
        energy = np.trace(partial_density_matrix.dot(Pn_system_only))
        print(energy)
        E_list.append(energy * gamma_l)
        
    else:
        qubitOp = anti_set_list[0]
        for PauliWord, const in qubitOp.terms.items():
            if PauliWord:
                Pauli_matrix = IBM_PauliWord(qubitOp, n_qubits, draw=False, reverse=False)
                exp_val = np.dot(ANSATZ_bra, Pauli_matrix.dot(ANSATZ_STATE))
                E_list.append(exp_val*const)
            else:
                E_list.append(const)

print('FCI expected =', new_FCI_Energy)    
print('measured E = ', sum(E_list))
print('success:', np.allclose(sum(E_list), new_FCI_Energy))

(-0.9996744019687643-4.826200566250184e-18j)
(-0.9994616002448109+4.723125160811947e-18j)
(-0.9996744019687644-3.6464503205124436e-18j)
(-0.999461600244811-6.807272115397356e-17j)

FCI expected = (-1.137283834488503+3.7121881080805557e-17j)
measured E =  (-1.1372838344885015-1.3163353307091132e-17j)
success: True
